## Setting

In [12]:
import pandas as pd
import sys
import os
import numpy as np
from clust.pipeline import data_pipeline
from clust.tool import data_statistical, data_clustering
from clust.transformation.splitDataByCycle import dataByCycle
import datetime
from config import influxdb_client as db_client

sys.path.append("../")
sys.path.append("../..")
sys.path.append("../../..")
sys.path.append("../../../..")

## Get Data

In [14]:
bk_name = 'farm_protectedHorticulture_westernVegetables_paprika'
ms_name = 'environment_knu_n'
tags = {'FRM_ID': 'F00012', 'ZN_ID': '1구역'}
start_time = '2021-07-01 00:00:00'
end_time = '2021-10-31 23:59:59'
# end_time = '2021-11-01 00:00:00'

In [15]:
data = db_client.get_data_by_time(start_time, end_time, bk_name, ms_name, tags)
data

,EXT_TMP,EXT_WIND_DIRECT,EXT_WIND_SPEED,INT_CO2,INT_REL_HD,INT_TMP,MAX_EXT_TMP,MIN_EXT_TMP
time,,,,,,,,
2021-07-01 00:00:00+00:00,19.50,90.0,0.00,558.75,80.33,22.25,20.2,19.7
2021-07-01 00:10:00+00:00,19.87,90.0,0.30,394.20,63.42,13.31,20.3,19.5
2021-07-01 00:20:00+00:00,19.75,90.0,0.31,395.00,63.82,13.22,20.0,19.5
2021-07-01 00:30:00+00:00,19.69,90.0,0.15,399.50,63.63,13.21,19.8,19.5
2021-07-01 00:40:00+00:00,19.72,90.0,0.04,399.80,63.53,13.23,19.9,19.6
...,...,...,...,...,...,...,...,...
2021-10-31 19:00:00+00:00,9.24,NaN,0.31,447.76,90.44,16.09,NaN,NaN
2021-10-31 20:00:00+00:00,8.00,NaN,0.02,472.06,89.82,15.96,NaN,NaN
2021-10-31 21:00:00+00:00,7.22,NaN,0.00,497.28,90.00,16.31,NaN,NaN


## Data Slicing

In [16]:
cycle_unit = 'day'
cycle_count = 1
cycle_data = dataByCycle.generate_cycle_dataset(data, cycle_unit, cycle_count)

In [17]:
if len(data.columns) == 1:
    data_col = str(data.columns)

## Test

In [18]:
selected_statistical = "MINMAX" # IQR

In [19]:
if selected_statistical == 'MINMAX':
    min_max_dict = {
        "EXT_TMP": {"min_val": 0, "max_val": 40},
        "EXT_WIND_DIRECT": {"min_val": 90, "max_val": 270},
        "EXT_WIND_SPEED": {"min_val": 0, "max_val": 10},
        "INT_CO2": {"min_val": 0, "max_val": 1000},
        "INT_REL_HD": {"min_val": 20, "max_val": 100},
        "INT_TMP": {"min_val": 0, "max_val": 40},
        "MAX_EXT_TMP": {"min_val": 0, "max_val": 40},
        "MIN_EXT_TMP": {"min_val": 0, "max_val": 40}
    }
        
elif selected_statistical == 'IQR':
    low_val = 0.2
    high_val = 0.8
    lower_val, upper_val = data_statistical.compute_IQR(data, low_val, high_val)

In [20]:
col_check = "one" # one, all, specific
limit_count = 10

outliers_dict={}
cleaned_dict={}

if col_check == 'specific':
    selected_column = data_col
    cycle_data = {key: df[selected_column] if isinstance(selected_column, list) else df[[selected_column]] for key, df in cycle_data.items()}
                
if selected_statistical == 'MINMAX':
    for key, df in cycle_data.items():
        outliers_dict[key], cleaned_dict[key] = data_statistical.min_max_filters(df, min_max_dict)
        
elif selected_statistical == 'IQR':
    lower_val, upper_val = data_statistical.compute_IQR(data, low_val, high_val)
    lower_val = pd.Series(lower_val).astype(float)
    upper_val = pd.Series(upper_val).astype(float)
    
    for key, df in cycle_data.items():
        outliers_dict[key], cleaned_dict[key] = data_statistical.data_IQR_filters(df, lower_val, upper_val)

limit_count = 10
outlier_data = data_statistical.check_outliers(outliers_dict, limit_count, col_check)
selected_outlier_data = [key for key, is_outlier in outlier_data.items() if is_outlier]

outlier_data = {str(k): bool(v) if isinstance(v, (bool, np.bool_)) else int(v) for k, v in outlier_data.items()}
selected_outlier_data = [str(k) for k in selected_outlier_data]


In [21]:
outlier_data

{'2021-07-01 00:00:00+00:00 - 2021-07-01 23:50:00+00:00 | slice_1': False,
 '2021-07-02 00:00:00+00:00 - 2021-07-02 23:50:00+00:00 | slice_2': False,
 '2021-07-03 00:00:00+00:00 - 2021-07-03 23:50:00+00:00 | slice_3': False,
 '2021-07-04 00:00:00+00:00 - 2021-07-04 23:50:00+00:00 | slice_4': False,
 '2021-07-05 00:00:00+00:00 - 2021-07-05 23:50:00+00:00 | slice_5': False,
 '2021-07-06 00:00:00+00:00 - 2021-07-06 23:50:00+00:00 | slice_6': False,
 '2021-07-07 00:00:00+00:00 - 2021-07-07 23:50:00+00:00 | slice_7': False,
 '2021-07-08 00:00:00+00:00 - 2021-07-08 23:50:00+00:00 | slice_8': False,
 '2021-07-09 00:00:00+00:00 - 2021-07-09 23:50:00+00:00 | slice_9': False,
 '2021-07-10 00:00:00+00:00 - 2021-07-10 23:50:00+00:00 | slice_10': False,
 '2021-07-11 00:00:00+00:00 - 2021-07-11 23:50:00+00:00 | slice_11': False,
 '2021-07-12 00:00:00+00:00 - 2021-07-12 23:50:00+00:00 | slice_12': False,
 '2021-07-13 00:00:00+00:00 - 2021-07-13 23:50:00+00:00 | slice_13': False,
 '2021-07-14 00:00:00

In [22]:
selected_outlier_data

['2021-10-17 00:00:00+00:00 - 2021-10-17 23:50:00+00:00 | slice_109',
 '2021-10-18 00:00:00+00:00 - 2021-10-18 23:50:00+00:00 | slice_110',
 '2021-10-21 00:00:00+00:00 - 2021-10-21 23:50:00+00:00 | slice_113',
 '2021-10-22 00:00:00+00:00 - 2021-10-22 23:50:00+00:00 | slice_114',
 '2021-10-23 00:00:00+00:00 - 2021-10-23 23:50:00+00:00 | slice_115',
 '2021-10-24 00:00:00+00:00 - 2021-10-24 23:50:00+00:00 | slice_116',
 '2021-10-25 00:00:00+00:00 - 2021-10-25 23:50:00+00:00 | slice_117',
 '2021-10-28 00:00:00+00:00 - 2021-10-28 23:50:00+00:00 | slice_120']